In [1]:
import pandas as pd
import math
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
dataset = pd.read_csv("Arrest_Data_from_2010_to_2019.csv")
dataset.head()

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location
0,191811472,05/03/2019,1700.0,18,Southeast,1802,23,F,B,NaN,NaN,M,653.22 PC,NaN,91ST,FIGUEROA,"(33.9543, -118.2827)"
1,5614161,04/29/2019,1040.0,8,West LA,842,41,M,H,3.0,Robbery,F,211PC,ROBBERY,11600 WILSHIRE BL,NaN,"(34.0508, -118.4592)"
2,5615197,04/30/2019,615.0,6,Hollywood,663,27,M,O,5.0,Burglary,F,459PC,BURGLARY,LA BREA,LEXINGTON,"(34.0907, -118.3384)"
3,5615701,04/30/2019,1100.0,9,Van Nuys,901,2,F,H,NaN,NaN,D,300(B)WIC,NaN,RAYMER,SEPULVEDA BL,"(34.2149, -118.4674)"
4,5615929,04/30/2019,1550.0,20,Olympic,2049,41,M,W,4.0,Aggravated Assault,F,245(A)(2)PC,ADW W/FIREARM,8TH ST,VERMONT,"(34.0578, -118.2916)"


In [3]:
len(dataset[dataset.isna().any(axis=1)])

618537

In [4]:
dataset.columns

Index(['Report ID', 'Arrest Date', 'Time', 'Area ID', 'Area Name',
       'Reporting District', 'Age', 'Sex Code', 'Descent Code',
       'Charge Group Code', 'Charge Group Description', 'Arrest Type Code',
       'Charge', 'Charge Description', 'Address', 'Cross Street', 'Location'],
      dtype='object')

In [5]:
dataset.isna().sum()

Report ID                        0
Arrest Date                      0
Time                           195
Area ID                          0
Area Name                        0
Reporting District               0
Age                              0
Sex Code                         0
Descent Code                     0
Charge Group Code            88029
Charge Group Description     88550
Arrest Type Code                 0
Charge                           0
Charge Description           87987
Address                          0
Cross Street                562466
Location                         0
dtype: int64

In [6]:
dataset[['LAT', 'LONG']] = dataset['Location'].str.strip('()').str.split(', ', expand=True)
dataset['LAT'] = dataset['LAT'].astype(float)
dataset['LONG'] = dataset['LONG'].astype(float)

In [7]:
dataset['Arrest Date'] = pd.to_datetime(dataset['Arrest Date'])

### Q1

In [8]:
arrested_2018 = dataset[(dataset['Arrest Date'] >= '2018-01-01') & (dataset['Arrest Date'] <= '2018-12-31')]
len(arrested_2018)

104277

### Q2

In [96]:
print(f"{arrested_2018['Area Name'].value_counts().idxmax()}: {arrested_2018['Area Name'].value_counts().max()}")

Central: 10951


### Q3

In [10]:
arrested_2018['Charge Group Description'].unique()

array(['Fraud/Embezzlement', 'Homicide', 'Narcotic Drug Laws',
       'Weapon (carry/poss)', 'Other Assaults',
       'Miscellaneous Other Violations', 'Vehicle Theft',
       'Aggravated Assault', 'Driving Under Influence', 'Robbery',
       'Burglary', 'Against Family/Child', 'Moving Traffic Violations',
       'Larceny', 'Forgery/Counterfeit', 'Receive Stolen Property',
       'Sex (except rape/prst)', 'Prostitution/Allied', 'Liquor Laws',
       'Drunkeness', 'Rape', 'Disturbing the Peace',
       'Non-Criminal Detention', nan, 'Disorderly Conduct', 'Gambling',
       'Pre-Delinquency', 'Federal Offenses'], dtype=object)

In [60]:
specific_charges = arrested_2018[
    (arrested_2018['Charge Group Description'] == 'Vehicle Theft') |
    (arrested_2018['Charge Group Description'] == 'Robbery') |
    (arrested_2018['Charge Group Description'] == 'Burglary') |
    (arrested_2018['Charge Group Description'] == 'Receive Stolen Property')
]
specific_charges.head(3)

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location,LAT,LONG
167,5449454,2018-10-15,2330.0,1,Central,157,43,M,H,7.0,Vehicle Theft,F,10851(A)VC,TAKE VEHICLE W/O OWNER'S CONSENT,SPRING,5TH,"(34.0416, -118.244)",34.0416,-118.2440
172,5449463,2018-10-15,2330.0,1,Central,154,32,M,H,7.0,Vehicle Theft,F,10851(A)VC,TAKE VEHICLE W/O OWNER'S CONSENT,SPRING ST,6TH,"(34.046, -118.2509)",34.0460,-118.2509
173,5452727,2018-10-19,1935.0,1,Central,100,26,M,B,7.0,Vehicle Theft,F,10851(A)VC,TAKE VEHICLE W/O OWNER'S CONSENT,8TH ST,LOS ANGELES,"(34.0413, -118.2517)",34.0413,-118.2517


In [12]:
quantile_95p = specific_charges['Age'].quantile(0.95)
quantile_95p

52.0

### Q4

In [13]:
charged_2018 = arrested_2018[(arrested_2018['Charge Group Description'] != 'nan') &
                            (arrested_2018['Charge Group Description'] != 'Pre-Delinquency') & 
                            (arrested_2018['Charge Group Description'] != 'Non-Criminal Detention') & 
                            (arrested_2018['Charge Group Description'].notna()) &
                            (arrested_2018['Charge Group Description'] != '')
                            ]
charged_2018.head()

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location,LAT,LONG
156,5213460,2018-01-24,1930.0,6,Hollywood,668,29,M,H,10.0,Fraud/Embezzlement,F,530.5(A)PC,GET CREDIT/ETC OTHER'S ID,ST ANDREWS,FOUNTAIN,"(34.0949, -118.3109)",34.0949,-118.3109
157,5257944,2018-03-15,310.0,7,Wilshire,743,33,F,O,1.0,Homicide,F,191.5(B)PC,VEH MANSLAUGHTR/OPER VESSEL DUI & DUR FEL,OLYMPIC,MASSELIN,"(34.0574, -118.3547)",34.0574,-118.3547
158,5328813,2018-06-01,1800.0,3,Southwest,356,44,F,B,16.0,Narcotic Drug Laws,F,11379(A)HS,TRANSPORT/SELL CONTROLLED SUBSTANCE,1600 W 36TH PL,NaN,"(34.0219, -118.3061)",34.0219,-118.3061
159,5336331,2018-06-10,200.0,13,Newton,1364,23,M,H,12.0,Weapon (carry/poss),F,25400(A)2PC,CARRYING CONCEALED WEAPON UPON PERSON,1200 E 50TH ST,NaN,"(33.998, -118.2543)",33.9980,-118.2543
160,5360731,2018-07-07,2330.0,18,Southeast,1832,32,M,B,12.0,Weapon (carry/poss),F,29800(A)1PC,POSS F/ARM BY CONVICTED FELON/ADDICT/ETC,10700 S MAIN ST,NaN,"(33.9392, -118.2739)",33.9392,-118.2739


In [14]:
average_ages = charged_2018.groupby('Charge Group Description')['Age'].mean()

std_ages = average_ages.std()
avg_ages = average_ages.mean()

z_values = []

for age in average_ages:
    z_values.append((age - avg_ages) / std_ages)

max_score = max(abs(z) for z in z_values)
round(max_score, 4)

2.1279

### Q5

In [15]:
arrested_2018.head()

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location,LAT,LONG
156,5213460,2018-01-24,1930.0,6,Hollywood,668,29,M,H,10.0,Fraud/Embezzlement,F,530.5(A)PC,GET CREDIT/ETC OTHER'S ID,ST ANDREWS,FOUNTAIN,"(34.0949, -118.3109)",34.0949,-118.3109
157,5257944,2018-03-15,310.0,7,Wilshire,743,33,F,O,1.0,Homicide,F,191.5(B)PC,VEH MANSLAUGHTR/OPER VESSEL DUI & DUR FEL,OLYMPIC,MASSELIN,"(34.0574, -118.3547)",34.0574,-118.3547
158,5328813,2018-06-01,1800.0,3,Southwest,356,44,F,B,16.0,Narcotic Drug Laws,F,11379(A)HS,TRANSPORT/SELL CONTROLLED SUBSTANCE,1600 W 36TH PL,NaN,"(34.0219, -118.3061)",34.0219,-118.3061
159,5336331,2018-06-10,200.0,13,Newton,1364,23,M,H,12.0,Weapon (carry/poss),F,25400(A)2PC,CARRYING CONCEALED WEAPON UPON PERSON,1200 E 50TH ST,NaN,"(33.998, -118.2543)",33.9980,-118.2543
160,5360731,2018-07-07,2330.0,18,Southeast,1832,32,M,B,12.0,Weapon (carry/poss),F,29800(A)1PC,POSS F/ARM BY CONVICTED FELON/ADDICT/ETC,10700 S MAIN ST,NaN,"(33.9392, -118.2739)",33.9392,-118.2739


In [16]:
changed_location = arrested_2018[(arrested_2018['LAT'] != 0) | (arrested_2018['LONG'] != 0)]

In [17]:
def distanceEarth(lat, long):
    original_lat, original_long = 34.050536, -118.247861
    
    del_lat = math.radians(original_lat - lat)
    del_long = math.radians(original_long - long)
    avg_lat = math.radians((original_lat + lat)) / 2
    r = 6371
    
    distance = r * math.sqrt((del_lat * math.cos(del_long/2))** 2 + (2 *math.cos(avg_lat) * math.sin(del_long/2))** 2)

    return round(distance, 2)


In [18]:
changed_location['Distance'] = changed_location.apply(lambda row: distanceEarth(row['LAT'], row['LONG']), axis=1)

In [19]:
distance_2km = changed_location[changed_location['Distance'] <= 2]

In [20]:
len(distance_2km)

11479

### Q6

In [21]:
picoList = dataset[dataset['Address'].str.contains('Pico', case=False, na=False)]

In [22]:
picoList = picoList[(picoList['LAT'] != 0) | (picoList['LONG'] != 0)]

In [99]:
def remove_outliers(lat, long):
    lat_mean = picoList['LAT'].mean()
    lat_std = picoList['LAT'].std()
    long_mean = picoList['LONG'].mean()
    long_std = picoList['LONG'].std()

    lat_lower = lat_mean -2 *lat_std
    lat_upper = lat_mean + 2 *lat_std
    long_lower = long_mean - 2 *long_std
    long_upper = long_mean +2 *long_std

    lat_correct = lat_lower <= lat <= lat_upper
    long_correct = long_lower <= long <= long_upper
    
    return lat_correct and long_correct



In [100]:
picoList['isOutlier'] = picoList.apply(lambda row: remove_outliers(row['LAT'], row['LONG']), axis=1)

In [101]:
western_ind = picoList['LONG'].idxmin()
eastern_ind = picoList['LONG'].idxmax()

western_coord = (picoList.loc[most_western_ind, 'LAT'], picoList.loc[most_western_ind, 'LONG'])
eastern_coord = (picoList.loc[most_eastern_ind, 'LAT'], picoList.loc[most_eastern_ind, 'LONG'])

In [102]:
del_lat = math.radians(eastern_coord[0] - western_coord[0])
del_long = math.radians(eastern_coord[1] - western_coord[1])
avg_lat = math.radians((eastern_coord[0] + western_coord[0])) / 2

r = 6371

distance = r * math.sqrt((del_lat * math.cos(del_long/2))** 2 + (2 *math.cos(avg_lat) * math.sin(del_long/2))** 2)

distance

37.428634907513676

In [27]:
incident_on_Pico = len(picoList[(picoList['Arrest Date'] >= '2018-01-01') & (picoList['Arrest Date'] <= '2018-12-31')])

In [28]:
incident_on_Pico/distance

16.377834818574755

### Q7

In [103]:
unique_val = dataset['Charge Group Code'].unique()
unique_val_list = unique_val.tolist()
print(unique_val_list)

[nan, 3.0, 5.0, 4.0, 24.0, 18.0, 6.0, 17.0, 16.0, 20.0, 13.0, 19.0, 22.0, 21.0, 14.0, 10.0, 11.0, 8.0, 1.0, 12.0, 7.0, 15.0, 9.0, 23.0, 26.0, 2.0, 25.0, 29.0, 27.0, 99.0]


In [61]:
corrected_charge = dataset[dataset['Charge Group Code'].notna() & (dataset['Charge Group Code'] != 99.0)]
corrected_charge.head(3)

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location,LAT,LONG
1,5614161,2019-04-29,1040.0,8,West LA,842,41,M,H,3.0,Robbery,F,211PC,ROBBERY,11600 WILSHIRE BL,NaN,"(34.0508, -118.4592)",34.0508,-118.4592
2,5615197,2019-04-30,615.0,6,Hollywood,663,27,M,O,5.0,Burglary,F,459PC,BURGLARY,LA BREA,LEXINGTON,"(34.0907, -118.3384)",34.0907,-118.3384
4,5615929,2019-04-30,1550.0,20,Olympic,2049,41,M,W,4.0,Aggravated Assault,F,245(A)(2)PC,ADW W/FIREARM,8TH ST,VERMONT,"(34.0578, -118.2916)",34.0578,-118.2916


In [112]:
totalArrestsCharge = corrected_charge['Charge Group Code'].value_counts()

In [113]:
arrestsAreaCharge = corrected_charge.groupby(['Area ID', 'Charge Group Code']).size().reset_index(name='Count')

In [114]:
arrestsAreaCharge['Cond_Prob'] = arrestsAreaCharge['Count']/arrestsAreaCharge.groupby('Area ID')['Count'].transform('sum')

In [115]:
arrestsAreaCharge.head(3)

,Area ID,Charge Group Code,Count,Cond_Prob
0,1,1.0,148,0.001193
1,1,2.0,215,0.001733
2,1,3.0,1723,0.013889


In [116]:
len(corrected_charge)

1222091

In [117]:
totalArrestsCharge = totalArrestsCharge.reset_index()
totalArrestsCharge.columns = ['Charge Group Code', 'Count']
totalArrestsCharge['Uncond_Prob'] = totalArrestsCharge['Count']/len(corrected_charge)

In [118]:
totalArrestsCharge.head(3)

,Charge Group Code,Count,Uncond_Prob
0,24.0,242661,0.198562
1,16.0,162327,0.132827
2,22.0,120206,0.098361


In [119]:
merged_data = pd.merge(arrestsAreaCharge, totalArrestsCharge, on='Charge Group Code')
merged_data['Ratio'] = merged_data['Cond_Prob']/merged_data['Uncond_Prob']

In [120]:
merged_data.head(3)

,Area ID,Charge Group Code,Count_x,Cond_Prob,Count_y,Uncond_Prob,Ratio
0,1,1.0,148,0.001193,2449,0.002004,0.595322
1,1,2.0,215,0.001733,2983,0.002441,0.710009
2,1,3.0,1723,0.013889,26697,0.021845,0.635772


In [122]:
top5Ratios = merged_data.nlargest(5, 'Ratio')
top5Ratios['Ratio'].mean()

3.4766713648060468

### Q8

In [148]:
felony_crime = dataset[dataset['Charge Description'].str.contains('Felony', case=False, na=False)]

In [149]:
felony_crime.head(3)

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location,LAT,LONG
580,5596594,2019-04-10,1150.0,1,Central,127,46,M,B,24.0,Miscellaneous Other Violations,F,3056PC,VIOLATION OF PAROLE:FELONY,100 N LOS ANGELES ST,NaN,"(34.0515, -118.2424)",34.0515,-118.2424
611,5597587,2019-04-11,1045.0,1,Central,158,31,M,H,24.0,Miscellaneous Other Violations,F,3056PC,VIOLATION OF PAROLE:FELONY,7TH,CERES,"(34.038, -118.243)",34.0380,-118.2430
702,5601042,2019-04-15,1115.0,6,Hollywood,656,30,M,B,24.0,Miscellaneous Other Violations,F,3056PC,VIOLATION OF PAROLE:FELONY,WILLOUGHBY,VINE,"(34.0871, -118.3266)",34.0871,-118.3266
